In [1]:
!which python
!python --version
!nvidia-smi

/home/projects/dharel/nadavt/.conda/envs/jaymody-sps-env/bin/python
Python 3.9.10
/bin/bash: nvidia-smi: command not found


In [2]:
# !python -m torch.utils.collect_env

In [3]:
pip install -q transformers

Note: you may need to restart the kernel to use updated packages.


In [4]:
import torch
from transformers import AutoTokenizer, GPT2LMHeadModel

tokenizer = AutoTokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")

inputs = tokenizer("Hello, my dog is cute", return_tensors="pt")
outputs = model(**inputs, labels=inputs["input_ids"])
loss = outputs.loss
logits = outputs.logits
logits.shape

/home/projects/dharel/nadavt/.conda/envs/jaymody-sps-env/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


torch.Size([1, 6, 50257])

In [5]:
inputs

{'input_ids': tensor([[15496,    11,   616,  3290,   318, 13779]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}

In [17]:
outputs = model.generate(**inputs, max_new_tokens=20, return_dict_in_generate=True, output_scores=True, do_sample=False)
print("====")
print("Outputs:")
print(outputs)
print("====")
transition_scores = model.compute_transition_scores(
    outputs.sequences, outputs.scores, normalize_logits=True
)
print("transition_scores:")
print(transition_scores)

Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


====
Outputs:
GreedySearchDecoderOnlyOutput(sequences=tensor([[15496,    11,   616,  3290,   318, 13779,    13,   314,  1101,   407,
          1654,   611,   673,   338,   257, 26188,   393,   407,    13,   314,
          1101,   407,  1654,   611,   673,   338]]), scores=(tensor([[-77.4425, -80.4462, -88.0497,  ..., -96.2564, -93.6345, -84.0666]]), tensor([[-142.3193, -142.1796, -144.0388,  ..., -155.6977, -153.8760,
         -137.7941]]), tensor([[-175.1097, -174.3555, -180.2891,  ..., -186.5559, -176.5305,
         -176.7950]]), tensor([[-149.5665, -149.3981, -153.9603,  ..., -158.9080, -155.3729,
         -151.3570]]), tensor([[-146.1682, -147.2101, -157.6489,  ..., -158.1571, -155.2080,
         -150.0087]]), tensor([[-139.9639, -140.1996, -149.4295,  ..., -151.1516, -150.9630,
         -143.5753]]), tensor([[-111.7724, -111.0601, -118.1015,  ..., -119.4616, -118.6231,
         -113.0703]]), tensor([[-144.3919, -142.3804, -151.1539,  ..., -153.2746, -146.0314,
         -144.4816]]

In [12]:
from typing import Any
from torch import Tensor
from collections.abc import Iterable


def print_shapes(obj: Any, level: int = 0):
    padding = "--" * level
    
    if isinstance(obj, Tensor):
        print(f"{padding}{obj.shape}")
    elif isinstance(obj, dict):
        for k, v in obj.items():
            print(f"{padding}{k}")
            print_shapes(v, level + 1)
    elif isinstance(obj, str):
        print(f"{padding}type {type(obj)} is not supported")
    elif isinstance(obj, Iterable):
        for x in obj:
            print_shapes(x, level + 1)
    else:
        print(f"{padding}type {type(obj)} is not supported")


# Test cases
print_shapes(["test", Tensor([1, 2, 3]), {"key": Tensor([4, 5, 6])}])
print("====")
print_shapes("This is a string")
print("====")
print_shapes((1, 2, Tensor([7, 8, 9]), {"nested": [Tensor([10, 11, 12]), "string"]}))


--type <class 'str'> is not supported
--torch.Size([3])
--key
----torch.Size([3])
====
type <class 'str'> is not supported
====
--type <class 'int'> is not supported
--type <class 'int'> is not supported
--torch.Size([3])
--nested
------torch.Size([3])
------type <class 'str'> is not supported


In [13]:
print_shapes(outputs)

sequences
--torch.Size([1, 26])
scores
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])
----torch.Size([1, 50257])


In [15]:
type(outputs.scores), len(outputs.scores)

(tuple, 20)

In [16]:
print(outputs.scores[-1].shape)
print(outputs.scores[-1].min())
print(outputs.scores[-1].max())

torch.Size([1, 50257])
tensor(-156.0660)
tensor(-112.9710)
